# Read All Dataset CSV

In [32]:
import os
import csv
import re
import pandas as pd
import numpy as np

In [33]:
dataset_names=[]
X_trains=[]
y_trains=[]
X_tests=[]

# 收集所有資料夾名稱，按照數字順序排序資料夾名稱
for folder_name in os.listdir("../Competition_data"):
    dataset_names.append(folder_name)
dataset_names = sorted(dataset_names, key=lambda x: int(re.search(r'\d+', x).group()))

for folder_name in dataset_names:
    # print(folder_name)
    X_trains.append(pd.read_csv(f"../Competition_data/{folder_name}/X_train.csv",header=0))
    y_trains.append(pd.read_csv(f"../Competition_data/{folder_name}/y_train.csv",header=0))
    X_tests.append(pd.read_csv(f"../Competition_data/{folder_name}/X_test.csv",header=0))
    
    
    
# print(len(dataset_names))
# print(len(X_trains))  # 49, 代表有 49 個 dataFrame (每個資料集各一個)
# print(len(y_trains))
# print(len(X_tests))
# print(X_trains[0].dtypes)
# print(y_trains[0].dtypes)

## Data Preprocessing & Feature Engineering
我們試了許多方法，如xgboost 選擇importance，PCA降維度， PYOD 的 IForest 檢測異常值，one hot encoding...等結果比什麼都不做來的差

## MODEL XGBoost為主要訓練model
然後把準確率低的值重新用CV 調參數


In [34]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import json
from sklearn.metrics import roc_auc_score

### 將大家的參數存入JSON中，

In [40]:
import json

# 加載現有的參數文件
try:
    with open('best_params.json', 'r') as file:
        best_params = json.load(file)
except FileNotFoundError:
    # 如果文件不存在，初始化一個空字典
    best_params = {}

for i in range(len(dataset_names)):
    best_params.setdefault(f"dataset_{i+1}", {"auc": 0, "params": {}})
    # 保存更新後的 JSON 文件
with open('best_params.json', 'w') as file:
    json.dump(best_params, file, indent=4)

In [41]:

models=[]
retrain_index = []
avg_auc = 0
avg_train = 0
smote = SMOTE(random_state=42)
bad_threshold = 0.7
params = {
    "n_estimators": 700,
    "eta": 0.008,
    "gamma": 0.95,
    "scale_pos_weight": 2,
    "max_depth": 15,
    "subsample": 0.7,
    "colsample_bytree": 0.6,
    "objective": "binary:logistic",
    "eval_metric": "auc",
    "reg_lambda": 1,            # L2 regularization term on weights
    "alpha": 0.1                # L1 regularization term on weights
}

for i in range(len(dataset_names)):
    tmp_X_train, tmp_X_test, tmp_y_train, tmp_y_test = train_test_split(
        X_trains[i], y_trains[i], test_size=0.2, random_state=42, 
    )
    tmp_X_train,tmp_y_train = smote.fit_resample(tmp_X_train, tmp_y_train)

    model = XGBClassifier( **params)
    model.fit(tmp_X_train, tmp_y_train.squeeze())
    
    
    tmp_y_prob = model.predict_proba(tmp_X_test)[:, 1]
    auc = roc_auc_score(tmp_y_test, tmp_y_prob)
    if(auc < bad_threshold):            # 如果 AUC 太低，就將這個資料集的 index 存起來，等等重新訓練
        retrain_index.append(i)
        print(f"\033[31mAUC for dataset {i+1} with tuned parameters: {auc}\033[0m") 
    else:
        print(f"AUC for dataset {i+1} with tuned parameters: {auc}")
    avg_auc += auc
    models.append(model)
    #將參數存入 JSON 文件
    if auc >= best_params[f"dataset_{i+1}"]["auc"]:
        best_params[f"dataset_{i+1}"]["auc"] = auc
        best_params[f"dataset_{i+1}"]["params"].update(params)
    with open('best_params.json', 'w') as file:
        json.dump(best_params, file, indent=4)

        
print(f"avg auc :   {avg_auc / len(dataset_names)}")



AUC for dataset 1 with tuned parameters: 0.8514246947082769
AUC for dataset 2 with tuned parameters: 0.9995215311004785
AUC for dataset 3 with tuned parameters: 0.7916666666666667
AUC for dataset 4 with tuned parameters: 0.44693877551020406
AUC for dataset 5 with tuned parameters: 0.9066666666666667
AUC for dataset 6 with tuned parameters: 0.9551020408163265
AUC for dataset 7 with tuned parameters: 0.9786780383795309
AUC for dataset 8 with tuned parameters: 0.9418181818181819
AUC for dataset 9 with tuned parameters: 0.8942307692307693
AUC for dataset 10 with tuned parameters: 0.7934731934731934
AUC for dataset 11 with tuned parameters: 0.2857142857142857
AUC for dataset 12 with tuned parameters: 0.9904761904761904
AUC for dataset 13 with tuned parameters: 0.96
AUC for dataset 14 with tuned parameters: 1.0
AUC for dataset 15 with tuned parameters: 0.7583333333333332
AUC for dataset 16 with tuned parameters: 0.9995215311004785
AUC for dataset 17 with tuned parameters: 0.9259259259259259


#### 把準確率低的抓出來重新訓練

考量要跑很久，我們有直接將訓練好的參數記錄在best_params.json

如果要自己逐步調整(一次調一兩個參數，要把下面xgboost 同樣名字的參數註解掉才不會衝突)

In [ ]:

# from sklearn.model_selection import GridSearchCV
# from sklearn.model_selection import StratifiedKFold
# param_grid = {
    
#     # 'learning_rate': [0.005, 0.01,0.05,0.1],
#     # 'n_estimators': [300,400,500],         #500 400 is mostly good 
#     # 'max_depth': [ 2,3,5,6, 7,9,10,12],
#     # 'subsample': [0.5, 0.6, 0.7, 0.8, 0.9],
#     # 'colsample_bytree': [0.5, 0.6, 0.7, 0.8, 0.9],
#     # 'alpha': [0.01,0.1, 0.5,1.5,],
#     # 'reg_lambda': [0.01,0.1, 0.5,2],
#     # 'gamma': [0.1,0.8, 0.95, 1, 1.5,],
#     # 'scale_pos_weight' : [1,2,3,5],
# }
# print(retrain_index)

# for i in retrain_index:
#     tmp_X_train, tmp_X_test, tmp_y_train, tmp_y_test = train_test_split(
#         X_trains[i], y_trains[i], test_size=0.2, random_state=42, 
#     )
#     tmp_X_train,tmp_y_train = smote.fit_resample(tmp_X_train, tmp_y_train)
#     model = XGBClassifier(
#     #     n_estimators=700, 
#     #     eta=0.008,
        
#     #     # gamma=0.95,    # 1 is best
#     #     min_child_weight = 2,
#     #     # scale_pos_weight=2,
#     #     max_depth=15, 
#     #     subsample=0.5, 
#     #     colsample_bytree=0.9,
#     #     objective='binary:logistic',
#     #     eval_metric='logloss',
#     #     # reg_lambda=1,     # L2 regularization term on weights
#     #     # alpha=0.1         # L1 regularization term on weights
#     **best_params.get(f"dataset_{i+1}", {}).get("params", {})  # Load the best parameters from the JSON file
#     )
        
    
#     cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
#     grid_search = GridSearchCV(model, param_grid, cv=cv, scoring='roc_auc', n_jobs=-1)
#     grid_search.fit(tmp_X_train, tmp_y_train.squeeze())
   
#     # Get the best model from cross-validation
#     best_model = grid_search.best_estimator_
    
#     print(f"Best parameters for dataset {i+1}: {grid_search.best_params_}")

#     # Evaluate the best model
#     tmp_y_prob = best_model.predict_proba(tmp_X_test)[:, 1]
   
#     auc = roc_auc_score(tmp_y_test, tmp_y_prob)
#     #Save the best parameters to JSON
#     # 更新或追加參數(if auc better)
#     # if auc >= best_params.get(f"dataset_{i+1}", {}).get('auc', 0):
#     #     print(f"\033[32mAUC for dataset {i+1} with tuned parameters: {auc}\033[0m")
#     #     best_params[f"dataset_{i+1}"]['auc'] = auc

#     #     if f"dataset_{i+1}" in best_params:
#     #         best_params[f"dataset_{i+1}"]["params"].update(grid_search.best_params_)
#     #     else:
#     #         best_params[f"dataset_{i+1}"]["params"]  = grid_search.best_params_
#     # else:
#     print(f"AUC for retrain dataset {i+1} with tuned parameters: {auc}")
#     # Save the best parameters to a JSON file
    
# with open('best_params.json', 'w') as file:
#     json.dump(best_params, file, indent=4)


In [ ]:
##最後重新train model by best params

# 讀取現有 JSON 文件
with open('best_params.json', 'r') as file:
    best_params = json.load(file)

models=[]
avg_auc = 0
avg_train = 0

for i in range(len(dataset_names)):
  
    tmp_X_train, tmp_X_test, tmp_y_train, tmp_y_test = train_test_split(
        X_trains[i], y_trains[i], test_size=0.2, random_state=42, 
    )
    tmp_X_train,tmp_y_train = smote.fit_resample(tmp_X_train, tmp_y_train)
    model = XGBClassifier(
        **best_params.get(f"dataset_{i+1}", {}).get("params", {}),  # Load the best parameters from the JSON file
    )
    model.fit(tmp_X_train, tmp_y_train.squeeze())
    
   
    tmp_y_prob = model.predict_proba(tmp_X_test)[:, 1]
    auc = roc_auc_score(tmp_y_test, tmp_y_prob)
    print(f'auc of dataset {i+1}: \t{auc}')
    avg_auc += auc
    models.append(model)
        
print(f"avg auc :   {avg_auc / len(dataset_names)}")
# print(f"avg auc of maxdeep of {max_deep}:   {avg_auc / len(dataset_names)}")


auc of dataset 1: 	0.8514246947082769
auc of dataset 2: 	0.9995215311004785
auc of dataset 3: 	0.7916666666666667
auc of dataset 4: 	0.44693877551020406
auc of dataset 5: 	0.9066666666666667
auc of dataset 6: 	0.9551020408163265
auc of dataset 7: 	0.9786780383795309
auc of dataset 8: 	0.9418181818181819
auc of dataset 9: 	0.8942307692307693
auc of dataset 10: 	0.7934731934731934
auc of dataset 11: 	0.37142857142857144
auc of dataset 12: 	0.9904761904761904
auc of dataset 13: 	0.96
auc of dataset 14: 	1.0
auc of dataset 15: 	0.7583333333333332
auc of dataset 16: 	0.9995215311004785
auc of dataset 17: 	0.9259259259259259
auc of dataset 18: 	1.0
auc of dataset 19: 	0.9920634920634921
auc of dataset 20: 	0.9575757575757575
auc of dataset 21: 	0.993006993006993
auc of dataset 22: 	0.869281045751634
auc of dataset 23: 	0.947695035460993
auc of dataset 24: 	0.6251709986320109
auc of dataset 25: 	1.0
auc of dataset 26: 	0.8461538461538461
auc of dataset 27: 	1.0
auc of dataset 28: 	0.856451612

## Inference Model

In [60]:
##給xgboost用
y_predicts=[]
for i in range(len(dataset_names)):
    # print(X_tests[i])
    y_predict_proba=models[i].predict_proba(X_tests[i])[:, 1]
    df = pd.DataFrame(y_predict_proba, columns=['y_predict_proba'])
    y_predicts.append(df)
    

## Save result

In [61]:
for idx,dataset_name in enumerate(dataset_names):
    df=y_predicts[idx]
    df.to_csv(f'../Competition_data/{dataset_name}/y_predict.csv', index=False,header=True)